In [1]:
!pip install geemap

In [2]:
import hydrofunctions as hf
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import folium
import geemap.foliumap as geemap
import ee
import matplotlib.dates as mdates
import hydrofns

%matplotlib inline

In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWjMCvH_QDiVS1rak6EmzmalI_5E-A4Do1I84RwHLFMa9WVzDGwhM7Q

Successfully saved authorization token.


In [4]:
# Site number for NWIS gage (Vicksburg MS Site)
vicksburg = ['07289000']

# Coordinates for site bounds at Vicksburg MS
vicksburg_xMin = -91.2
vicksburg_yMin = 32
vicksburg_xMax = -90.8
vicksburg_yMax = 32.7

# Generate Vicksburg site bounds geometry
vicksburg_rectangleGeoJSON = ee.Geometry.Rectangle(
    [
        [vicksburg_xMin, vicksburg_yMin],
        [vicksburg_xMax, vicksburg_yMax]])

vicksburg_bnds = ee.FeatureCollection(ee.Feature(vicksburg_rectangleGeoJSON,
                            {'name': 'Mississippi Vicksburg', 'fill': 1}))

# Palettes for Visualization
trueColor_palette = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
waterPalette = ['red', 'yellow', 'green', 'blue']

# Import the Sentinel 2 collection
sentinel2 = ee.ImageCollection(
    'COPERNICUS/S2_SR').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))

# Vicksburg MS wet and dry seasons
vicksburg_wet_season_start = '2019-03-15'
vicksburg_wet_season_end = '2019-05-01'

vicksburg_dry_season_start = '2019-9-01'
vicksburg_dry_season_end = '2019-10-01'

In [5]:
# Create images for wet and dry seasons at Vicksburg MS
vicksburg_dry_season = sentinel2.filterBounds(vicksburg_bnds).filterDate(
    vicksburg_dry_season_start, vicksburg_dry_season_end).mosaic().clip(vicksburg_bnds)

vicksburg_wet_season = sentinel2.filterBounds(vicksburg_bnds).filterDate(
    vicksburg_wet_season_start, vicksburg_wet_season_end).mosaic().clip(vicksburg_bnds)

In [6]:
# Create true color dry season image for Vicksburg MS
vicksburg_true_color_dry = geemap.Map(center=[32.35, -90.9], zoom=10)
vicksburg_true_color_dry.addLayer(vicksburg_dry_season, trueColor_palette,
             '2019 Dry season true color')
vicksburg_true_color_dry.addLayerControl()
vicksburg_true_color_dry

In [7]:
# Create true color wet season image for Vicksburg MS
vicksburg_true_color_wet = geemap.Map(center=[32.35, -90.9], zoom=10)
vicksburg_true_color_wet.addLayer(vicksburg_wet_season, trueColor_palette,
             '2019 Wet season true color')
vicksburg_true_color_wet.addLayerControl()
vicksburg_true_color_wet

In [8]:
# Create NDWI dry season image for Vicksburg MS
vicksburg_ndwi_map_dry = geemap.Map(center=[32.35, -90.9], zoom=10)
vicksburg_ndwi_dry = vicksburg_dry_season.normalizedDifference(['B3', 'B8'])
vicksburg_ndwi_map_dry.addLayer(vicksburg_ndwi_dry, {
                                  'min': -1, 'max': 0.5, 'palette': waterPalette}, '2019 Dry season NDWI')
vicksburg_ndwi_map_dry

In [9]:
# Create NDWI wet season image for Vicksburg MS
vicksburg_ndwi_map_wet = geemap.Map(center=[32.35, -90.9], zoom=10)
vicksburg_ndwi_wet = vicksburg_wet_season.normalizedDifference(['B3', 'B8'])
vicksburg_ndwi_map_wet.addLayer(vicksburg_ndwi_wet, {
                                  'min': -1, 'max': 0.5, 'palette': waterPalette}, '2019 Wet season NDWI')
vicksburg_ndwi_map_wet

In [10]:
# Create dry season water map for Vicksburg MS
vicksburg_water_map_dry = geemap.Map(center=[32.35, -90.9], zoom=10)

ndwi_threshold = 0
vicksburg_water_image_dry = vicksburg_ndwi_dry.gt(ndwi_threshold).selfMask()

vicksburg_water_map_dry.addLayer(vicksburg_water_image_dry, {
                                  'palette': 'blue'}, 'Water image')
vicksburg_water_map_dry

In [11]:
# Create wet season water map for Vicksburg MS
vicksburg_water_map_wet = geemap.Map(center=[32.35, -90.9], zoom=10)

ndwi_threshold = 0
vicksburg_water_image_wet = vicksburg_ndwi_wet.gt(ndwi_threshold).selfMask()

vicksburg_water_map_wet.addLayer(vicksburg_water_image_wet, {
                                  'palette': 'blue'}, 'Water image')
vicksburg_water_map_wet